# Build Your First AI Agent with Google ADK and Teradata (Part 1)

## Setting Up the Project 

This section outlines the steps required to set up the project:  
- Install Google ADK and import necessary libraries  
- Setup the required API keys and associated values
- Establish the database connection  
- Load the sample data into the Teradata database  


In [ ]:
# Install agent development kit
!pip install google-adk

In [ ]:
from google.adk.agents import LlmAgent, Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from teradataml import *
import pandas as pd

In [ ]:
# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "Your_api_key" # <--- REPLACE

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Agent model
AGENT_MODEL = "gemini-2.0-flash-001"

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
data_loading_queries = [
'''
CREATE DATABASE energy_consumption
AS PERMANENT = 50000000;
''',
'''
CREATE TABLE energy_consumption.march AS
(
  SELECT usage_date, kwh_consumed
     FROM (
		LOCATION='/s3/dev-rel-demos.s3.amazonaws.com/google_adk/energy_usage_march_2025.csv') as consumption
) WITH DATA;
'''
]
for query in data_loading_queries:
    execute_sql(query)

# Configuring the Agent

This section contains configuration of agent:
- Create a tool to query data
- Define the Agent
- Create session and runner

In [ ]:
def get_consumption_data(month: str) -> list[dict]:
    """Get the data for the whole month"""
    
    print(f"---Tool: get_consumption_data called for: {month}---")
    month_normalized = month.lower().replace(" ","")
    tables_df = db_list_tables(schema_name="energy_consumption")
    
    if month_normalized in tables_df["TableName"].to_list():
        query = '''select * from energy_consumption.%s''' %month_normalized
        td_df = DataFrame.from_query(query)
        df = td_df.to_pandas()
        df['usage_date'] = pd.to_datetime(df['usage_date']).dt.strftime('%d-%m-%Y')
        
        # Convert to list of dicts
        records = df.to_dict(orient='records')
        return records
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have energy consumption data of '{month}'month."}

# (Optional) Test the tool
# get_consumption_data("March")

In [ ]:
energy_analytics_agent = Agent(
    name = "energy_analytics_agent",
    model = AGENT_MODEL,
    description = (
        "Agent to analyse the energy consumption in a month"
        ),
    instruction = (
        "You are an Energy Analytics Agent. When a user provides a month name, you must:"
        "1. Invoke the tool 'get_consumption_data' to fetch daily energy usage for that month."
        "2. Perform whatever the user requests, such as:"
        "   • Retrieving a specific day’s consumption"
        "   • Calculating weekly or monthly totals, mean, and average"
        "   • Identifying spikes (days >20% above average) and drops (days >20% below average)"
        "   • Analyzing correlations between weekdays and weekends"
        "3. Draw at least two actionable insights or recommendations (e.g., possible causes of spikes, efficiency tips)."
        "4. Respond in Markdown, organized into these sections:"
        "   **Summary Statistics**, **Anomalies Detected**, **Insights & Recommendations**."
        ),
    tools = [get_consumption_data],
    output_key = "internal_data"
    )

In [ ]:
# --- Constants ---
APP_NAME = "energy_analytics_app"
USER_ID = "123"
SESSION_ID = "123001"

# --- Session Management ---
session_service = InMemorySessionService()
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
runner = Runner(
    agent=energy_analytics_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

# Running the Agent

This section handles the communication with Agent and presents the response to the user:
- Create function to handle user query asynchornously
- Define user query function


In [ ]:
import asyncio
from google.genai import types

async def call_agent_async(query: str):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")
    
    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    final_response_text = "Agent did not produce a final response." # Default

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            break
            
    print(f"<<< Agent Response: {final_response_text}")

In [ ]:
async def run_conversation():
    # await call_agent_async("What is the total power consumption in March?")
    # await call_agent_async("How about April?") # Expecting the tool's error message
    await call_agent_async("Tell me about the energy consumption of March")

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# Cleaning the testing data

In [ ]:
data_cleaning_queries = [
'''
DELETE DATABASE energy_consumption ALL;
''',
'''
DROP DATABASE energy_consumption
'''
]
for query in data_cleaning_queries:
    execute_sql(query)